In [2]:
import tensorflow as tf
from tensorflow import keras

In [5]:
!pip install -q -U keras-tuner

In [4]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [6]:
import keras_tuner as kt

In [7]:
(img_train,label_train),(img_test,label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [8]:
#normalizacja danych
img_train =img_train.astype('float32')/255.0
img_test =img_test.astype('float32')/255.0

In [9]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))

    hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
    model.add(keras.layers.Dense(units=hp_units,activation='relu'))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])


    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model


In [10]:
#utworzenie wystąpienia tunera i hypertuning
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [11]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
#wyszukiwanie hiperparametrów
tuner.search(img_train,label_train,epochs=50,validation_split=0.2,callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
        wyszukanie hiparparametrów zakończone.\n
        Optymalna liczba neuronów warstwy ukrytej: {best_hps.get('units')}\n
        Optymalna wartość learning rate: {best_hps.get('learning_rate')}

""")

Trial 4 Complete [00h 00m 22s]
val_accuracy: 0.8494166731834412

Best val_accuracy So Far: 0.8607500195503235
Total elapsed time: 00h 01m 25s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
480               |128               |units
0.0001            |0.001             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
1500/1500 [==============================] - 9s 6ms/step - loss: 0.6278 - accuracy: 0.7984 - val_loss: 0.4713 - val_accuracy: 0.8369
Epoch 2/2
1500/1500 [==============================] - 8s 6ms/step - loss: 0.4372 - accuracy: 0.8503 - val_loss: 0.4169 - val_accuracy: 0.8551
